In [2]:
!pip install folium

  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached folium-0.20.0-py2.py3-none-any.whl (113 kB)

   -------------------- ------------------- 1/2 [folium]
   -------------------- ------------------- 1/2 [folium]
   -------------------- ------------------- 1/2 [folium]
   ---------------------------------------- 2/2 [folium]



In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [6]:
# Download the CSV using pandas (for local jupyter notebook started from Anaconda)
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

print(f"Data loaded successfully!")
print(f"Shape: {spacex_df.shape}")
print(f"Columns: {spacex_df.columns.tolist()}")
print("\nFirst few rows:")
spacex_df.head()

Data loaded successfully!
Shape: (56, 13)
Columns: ['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site', 'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome', 'class', 'Lat', 'Long']

First few rows:


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [10]:
# For each launch site, add a circle and marker
for index, site in launch_sites_df.iterrows():
    site_coordinate = [site['Lat'], site['Long']]
    
    # Create a circle
    circle = folium.Circle(
        site_coordinate, 
        radius=1000, 
        color='#000000', 
        fill=True
    ).add_child(folium.Popup(site['Launch Site']))
    
    # Create a marker
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
        )
    )
    
    # Add to map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

All launch site are in proximity to the coast for safety as rockets fly over open water and if anything happens that's not a proplme
Optimized trajectory, fly eastward to take advantage of earth rotation
FAA and other regulation require clear launch corridors

In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [12]:
marker_cluster = MarkerCluster()

In [13]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Create marker_color column based on class value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

In [14]:
#TODO: For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# For each row in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Create a marker object with its coordinate
    coordinate = [record['Lat'], record['Long']]
    
    # Customize the Marker's icon property
    marker = folium.Marker(
        coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    # Add marker to marker_cluster
    marker_cluster.add_child(marker)

site_map

In [15]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
# Choose a launch site 
launch_site_lat = 28.56367
launch_site_lon = -80.57163

# Choose the closest coastline point 
coastline_lat = 28.56320
coastline_lon = -80.56750

# Calculate the distance
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a marker on the coastline with the distance displayed
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)

# Add it to the map
site_map.add_child(distance_marker)
site_map

In [18]:
# Create a line between launch site and coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')
site_map.add_child(lines)
site_map

In [19]:
# LC-39A launch site coordinates
launch_site_lat = 28.573255 
launch_site_lon = -80.646895  

# Titusville city coordinates
titusville_lat = 28.612218  
titusville_lon = -80.807553  

# Calculate distance to Titusville
distance_titusville = calculate_distance(launch_site_lat, launch_site_lon, titusville_lat, titusville_lon)

# Create Titusville marker with distance
titusville_marker = folium.Marker(
    [titusville_lat, titusville_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_titusville),
    )
)

# Create line from LC-39A to Titusville
titusville_coordinates = [[launch_site_lat, launch_site_lon], [titusville_lat, titusville_lon]]
titusville_line = folium.PolyLine(locations=titusville_coordinates, weight=1, color='green')

# Add to map
site_map.add_child(titusville_marker)
site_map.add_child(titusville_line)
site_map

In [20]:
# Railway coordinates near LC-39A
railway_lat = 28.578
railway_lon = -80.655

# Calculate distance
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Add railway marker
railway_marker = folium.Marker(
    [railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    )
)

# Add railway line
railway_coordinates = [[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]]
railway_line = folium.PolyLine(locations=railway_coordinates, weight=1, color='brown')

site_map.add_child(railway_marker)
site_map.add_child(railway_line)
site_map

In [21]:
# Highway coordinates near LC-39A (FL-405)
highway_lat = 28.580  
highway_lon = -80.650

# Calculate distance
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Add highway marker
highway_marker = folium.Marker(
    [highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#0000FF;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    )
)

# Add highway line
highway_coordinates = [[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]]
highway_line = folium.PolyLine(locations=highway_coordinates, weight=1, color='blue')

site_map.add_child(highway_marker)
site_map.add_child(highway_line)
site_map

Findings:
Safety
Coastal proximity is necessary for orbital launches (over-water trajectory)

City distance is important (10+ km buffers for populated areas)

Infrastructure proximity is required (rail/highway close but not too close)

